<a href="https://colab.research.google.com/github/Nigouz/3204-CourseWork2/blob/KNN/CourseWork2_Latest_Nicside.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
import re
from sklearn import preprocessing

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#FOR COURSEWORK
%matplotlib inline
plt.style.use('ggplot')
full_df = pd.read_csv(r"shuffled-noIPV6.csv")
# df = pd.read_csv("truncated_training.csv")

print(f"[*] Shape of dataset: {full_df.shape}")
from sklearn.utils import shuffle
full_df.drop(full_df.columns[0], axis=1, inplace=True)
full_df = shuffle(full_df)

print(full_df.tail())
print("\n")

In [ ]:
#COURSEWORK
full_df["src_port"]=  full_df["src_port"].replace(regex=",", value= "")
full_df["src_port"]=  full_df["src_port"].replace(regex=",", value= "")
full_df["dst_port"]=  full_df["dst_port"].replace(regex=",", value= "")
full_df["dst_port"]=  full_df["dst_port"].replace(regex=" ", value= "")
full_df["src_port"] = full_df["src_port"].replace(regex=" ", value="")
full_df["src_port"] = full_df["src_port"].replace(regex="dns", value="53")
full_df["src_port"] = full_df["src_port"].replace(regex="tls", value="0")
full_df["dst_port"] = full_df["dst_port"].replace(regex="dns", value="53")
full_df["dst_ip"] = full_df["dst_ip"].replace(regex="\S*:+\S+", value="0")

In [ ]:
"""Lab Files""" 
data4 = pd.read_csv("IanArffDataset.csv", delimiter=",")
#print("test1", data4)
d4 = data4.replace('?', 0)
d4.head()
print(f"[*] Shape of dataset: {d4.shape}")
d4.tail()
d4 = d4.dropna()

[*] Shape of dataset: (188508, 20)


In [ ]:
"""For LAB 8 """
#Data pre-processing
d4 = d4.rename(columns={"'address'": "address", "'function'": "function"})
d4 = d4.rename(columns={"'length'": "length", "'setpoint'": "setpoint", "'gain'": "gain", "'reset": "reset", "'deadband'": "deadband", "'cycle": "cycle"})
d4 = d4.rename(columns={"'rate'": "rate", "'system": "system", "'control": "control", "'pump'": "pump", "'solenoid'": "solenoid", "'pressure": "pressure"})
d4 = d4.rename(columns={"'crc": "crc", "'command response'": "command response", "'time'": "time", "'binary result'": "binary result", "'categorized": "categorized", "'specific": "specific"})

#print ("test3", d4)
#convert columns with object type to float64
d4 = d4.astype({'setpoint':'float64', 'gain':'float64', 'reset':'float64', 'deadband':'float64', 'cycle':'float64', 'rate':'float64', 'system':'float64', 'control':'float64', 'pump':'float64', 'solenoid':'float64', 'pressure':'float64'})

feature_d4 = d4[['address', 'function', 'length', 'setpoint', 'gain', 'reset', 'deadband', 'cycle', 'rate', 'system', 'control', 'pump', 'solenoid', 'pressure', 'crc', 'command response']]
X = np.asarray(feature_d4)
#print ("test5", X)

Y = d4['binary result']
print('x shape=', X.shape)
print('y shape=', Y.shape)

X = preprocessing.StandardScaler().fit(X).transform(X.astype(float)) #specifically for knn
#print (X)

In [ ]:
"Lab 8 training"
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [ ]:
from sklearn import metrics
scores = {}
scores_list= []

def scoring_metrics(y_test, y_pred, model):
    print(f"y_test size:{y_test.size} y_pred size:{y_pred.size}")
    accuracy = metrics.accuracy_score(y_test, y_pred)
    scores[model] = accuracy
    scores_list.append(accuracy)
#     confusion_matrix = metrics.confusion_matrix(y_true=[True, True], y_pred=[True, True], labels=[True, False])
    #For the Coursework its the following line
    cm = confusion_matrix(y_test, y_pred, labels=['nmap_scan', 'port_scan', 'smtp_enumeration', 'sql_enumeration', 'web_enumeration'])
    """For LAB 8"""
    #cm = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix: {cm}")

    print(f"\n[*] Model: {model}")
    print("[*] Precision: {:.3f}%".format(metrics.precision_score(y_test, y_pred, average="weighted")))
    print("[*] Recall: {:.3f}%".format(metrics.recall_score(y_test, y_pred, average="weighted")))

    print("[*] Accuracy: {:.3f}%".format(accuracy))
    print("[*] F1_score: {:.3f}%".format(metrics.f1_score(y_test, y_pred, average="weighted")))


# KNN

In [ ]:
df = full_df.head(20000)
# df = full_df

In [ ]:
print("Doing")
from sklearn.preprocessing import OneHotEncoder
df.replace(to_replace=["None"], value=np.nan, inplace=True)
clean_df = df.fillna(str(0)) #uhm i dont knowhow to explain this, but please lmk another alternative to fix this cause :") try to run it without str and you will know what i mean"
clean_x = clean_df.iloc[:, :13].values
clean_y = clean_df["category"].values
features = df.columns.values[:-1]

for label in clean_df.columns:
    for index, rows in clean_df.iterrows():
        new_ip = ""
        ip = str(rows[label])
        if re.search("\d+\.\d+\.\d+\.\d+", ip):
            octets = ip.split(".")
            for octet in octets:
                octet = octet.rjust(3,"0")
                new_ip += octet
            clean_df[label][index] = new_ip

clean_df["http_response_code"] = clean_df["http_response_code"].replace('HTTP/1.1"', value="0")
clean_df["src_ip"] = clean_df["src_ip"].replace('::1', value="0")
clean_df["dst_ip"] = clean_df["dst_ip"].replace('::1', value="0")
# clean_df["http_response_code"] = clean_df["http_response_code"].replace('HTTP/1.1"', value="0")
print("Done")

Doing


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Done


In [ ]:
from sklearn.compose import make_column_transformer
clean_x = clean_df.iloc[:, :13] #honestly don't know if this is correct lmaoooo
#this is to iterate through the columns and convert the strings to float (passthrough means for those columns we didnt specify, leave it as it is)
column_trans = make_column_transformer((OneHotEncoder(sparse=False), ['Protocol', 'http_request_method', 'http_request_referrer', 'url_path', 'user_agent_original', 'sql_method', 'sql_query']),remainder='passthrough')
test = column_trans.fit_transform(clean_x) #this is technically our cl

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
x_train, x_test, y_train, y_test = train_test_split(test, clean_y, test_size=0.3)

TypeError: ignored

In [ ]:
# k = 5
# knn = KNeighborsClassifier(n_neighbors=k)
# knn.fit(x_train, y_train)
# print(f"Saving trained model to test-knn{k}.sav")
# pickle.dump(knn, open(f"test-knn{k}.sav", "wb"))
# print("Model Saved")

# print("Loading knn from pickle")
# knn = pickle.load(open("knn5.sav", "rb"))
# print("KNN Successfully loaded")

# y_pred = knn.predict(x_test)
# scoring_metrics(y_test, y_pred, f"knn {k}")

In [ ]:
scores = {}
scores_list= []

range_k = range(3, 5)
knn = ""
skip = False


for k in range_k:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)

    y_pred = knn.predict(x_test)
    scoring_metrics(y_test, y_pred, f"knn {k}")

y_test size:56553 y_pred size:56553
Confusion Matrix: [[41802  2312]
 [ 6712  5727]]

[*] Model: knn 3
[*] Precision: 0.829%
[*] Recall: 0.840%
[*] Accuracy: 0.840%
[*] F1_score: 0.827%
y_test size:56553 y_pred size:56553
Confusion Matrix: [[43042  1072]
 [ 7411  5028]]

[*] Model: knn 4
[*] Precision: 0.847%
[*] Recall: 0.850%
[*] Accuracy: 0.850%
[*] F1_score: 0.829%


In [ ]:
# y_pred.tofile("y_pred", sep=",")
# y_test.tofile("y_test", sep=",")


In [ ]:
from sklearn.metrics import confusion_matrix
cf_test = confusion_matrix(y_test, y_pred, labels=['nmap_scan', 'port_scan', 'smtp_enumeration', 'sql_enumeration', 'web_enumeration'])

In [ ]:
y_test


array(['nmap_scan', 'port_scan', 'port_scan', ..., 'port_scan',
       'port_scan', 'web_enumeration'], dtype=object)